In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import spatialpy
from spatialpy import Model, Species, Parameter, Reaction, Mesh,\
                      PlaceInitialCondition, \
                      UniformInitialCondition, \
                      ScatterInitialCondition

# GeneRegulatoryNetwork2

In [2]:
class GeneRegulatoryNetwork2(Model):
    def __init__(self):
        Model.__init__(self, name="GeneRegulatoryNetwork2")

        # Domain
        mesh = Mesh.read_stochss_domain('GeneRegulatoryNetwork2.smdl')
        self.add_mesh(mesh)

        self.staticDomain = True

        # Variables
        G_F = Species(name="G_F", diffusion_constant=1e-12)
        G_O = Species(name="G_O", diffusion_constant=1e-12)
        P = Species(name="P", diffusion_constant=1e-12)
        mRNA = Species(name="mRNA", diffusion_constant=1e-12)
        self.add_species([G_F, G_O, P, mRNA])

        self.restrict(G_F, [])
        self.restrict(G_O, [])

        # Initial Conditions
        self.add_initial_condition(PlaceInitialCondition(species=G_F, count=1, location=[0, 0, 0]))

        # Parameters
        mu = Parameter(name="mu", expression="0.002")
        kappa = Parameter(name="kappa", expression="0.015")
        ka = Parameter(name="ka", expression="10000000")
        kd = Parameter(name="kd", expression="0.01")
        gamma_m = Parameter(name="gamma_m", expression="0.0006")
        gamma_p = Parameter(name="gamma_p", expression="0.0003")
        self.add_parameter([mu, kappa, ka, kd, gamma_m, gamma_p])

        # Reactions
        r1 = Reaction(name="r1", reactants={G_F: 1}, products={G_F: 1, mRNA: 1}, rate=self.listOfParameters["mu"])
        r2 = Reaction(name="r2", reactants={G_O: 1}, products={G_F: 1, P: 1}, rate=self.listOfParameters["ka"])
        r3 = Reaction(name="r3", reactants={G_F: 1, P: 1}, products={G_O: 1}, rate=self.listOfParameters["kd"])
        r4 = Reaction(name="r4", restrict_to=[2], reactants={mRNA: 1}, products={mRNA: 1, P: 1}, rate=self.listOfParameters["kappa"])
        r5 = Reaction(name="r5", reactants={mRNA: 1}, products={}, rate=self.listOfParameters["gamma_m"])
        r6 = Reaction(name="r6", reactants={P: 1}, products={}, rate=self.listOfParameters["gamma_p"])
        self.add_reaction([r1, r2, r3, r4, r5, r6])

        # Timespan
        self.timespan(np.arange(0, 1000, 10), timestep_size=10)

In [3]:
model = GeneRegulatoryNetwork2()

# Simulation Parameters

In [4]:
def configure_simulation():
    kwargs = {
        # "number_of_trajectories":1,
        # "seed":None,
    }
    return kwargs

In [5]:
kwargs = configure_simulation()
results = model.run(**kwargs)

# Visualization

In [ ]:
results.plot_species('G_F', animated=True, width=None, height=None)